# FunctionTask

In [1]:
import nest_asyncio

nest_asyncio.apply()

A `FunctionTask` is a `Task` that can be created from every *python* function by using *pydra* decorator: `pydra.mark.task`:

In [2]:
import pydra


@pydra.mark.task
def add_var(a, b):
    return a + b

Once we decorate the function, we can create a pydra `Task` and specify the input:

In [3]:
task1 = add_var(a=4, b=5)

We can check the type of `task1`:

In [4]:
type(task1)

pydra.engine.task.FunctionTask

and we can check if the task has correct values of `a` and `b`, they should be saved in the task `inputs`:

In [5]:
print(f'a = {task1.inputs.a}')
print(f'b = {task1.inputs.b}')

a = 4
b = 5


We can also check content of entire `inputs`:

In [6]:
task1.inputs

Inputs(a=4, b=5, _func=b'\x80\x05\x95\xba\x01\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x94\x8c\x0e_make_function\x94\x93\x94(h\x00\x8c\r_builtin_type\x94\x93\x94\x8c\x08CodeType\x94\x85\x94R\x94(K\x02K\x00K\x00K\x02K\x02K\x03C\x0c\x97\x00|\x00|\x01z\x00\x00\x00S\x00\x94N\x85\x94)\x8c\x01a\x94\x8c\x01b\x94\x86\x94\x8c /tmp/ipykernel_6421/214681761.py\x94\x8c\x07add_var\x94h\x0eK\x04C\x0b\x80\x00\xe0\x0b\x0c\x88q\x895\x80L\x94C\x00\x94))t\x94R\x94}\x94(\x8c\x0b__package__\x94N\x8c\x08__name__\x94\x8c\x08__main__\x94uNNNt\x94R\x94h\x00\x8c\x12_function_setstate\x94\x93\x94h\x18}\x94}\x94(h\x15h\x0e\x8c\x0c__qualname__\x94h\x0e\x8c\x0f__annotations__\x94}\x94\x8c\x0e__kwdefaults__\x94N\x8c\x0c__defaults__\x94N\x8c\n__module__\x94h\x16\x8c\x07__doc__\x94N\x8c\x0b__closure__\x94N\x8c\x17_cloudpickle_submodules\x94]\x94\x8c\x0b__globals__\x94}\x94u\x86\x94\x86R0.')

As you could see, `task.inputs` contains also information about the function, that is an inseparable part of the `FunctionTask`.

Once we have the task with set input, we can run it. Since `Task` is a "callable object", we can use the syntax:

In [7]:
task1()

Result(output=Output(out=9), runtime=None, errored=False)

As you can see, the result was returned right away, but we can also access it later:

In [8]:
task1.result()

Result(output=Output(out=9), runtime=None, errored=False)

`Result` contains more than just an output, so if we want to get the task output, we can type:

In [9]:
result = task1.result()
result.output.out

9

And if we want to see the input that was used in the task, we can set an optional argument `return_inputs` to True.

In [10]:
task1.result(return_inputs=True)

({'add_var.a': 4, 'add_var.b': 5},
 Result(output=Output(out=9), runtime=None, errored=False))

## Customizing output names
Note, that "out" is the default name for the task output, but we can always customize it. There are two ways of doing it: using *python* function annotation and using another *pydra* decorator:

Let's start from the function annotation:

In [11]:
import typing as ty


@pydra.mark.task
def add_var_an(a, b) -> {'sum_a_b': int}:
    return a + b


task1a = add_var_an(a=4, b=5)
task1a()

Result(output=Output(sum_a_b=9), runtime=None, errored=False)

The annotation might be very useful to specify the output names when the function returns multiple values.

In [12]:
@pydra.mark.task
def modf_an(a) -> {'fractional': ty.Any, 'integer': ty.Any}:
    import math

    return math.modf(a)


task2 = modf_an(a=3.5)
task2()

Result(output=Output(fractional=0.5, integer=3.0), runtime=None, errored=False)

The second way of customizing the output requires another decorator - `pydra.mark.annotate`

In [13]:
@pydra.mark.task
@pydra.mark.annotate({'return': {'fractional': ty.Any, 'integer': ty.Any}})
def modf(a):
    import math

    return math.modf(a)


task2a = modf(a=3.5)
task2a()

Result(output=Output(fractional=0.5, integer=3.0), runtime=None, errored=False)

**Note, that the order of the pydra decorators is important!**

## Setting the input

We don't have to provide the input when we create a task, we can always set it later:

In [14]:
task3 = add_var()
task3.inputs.a = 4
task3.inputs.b = 5
task3()

Result(output=Output(out=9), runtime=None, errored=False)

If we don't specify the input, `attr.NOTHING` will be used as the default value

In [15]:
task3a = add_var()
task3a.inputs.a = 4

# importing attr library, and checking the type of `b`
import attr

task3a.inputs.b == attr.NOTHING

True

And if we try to run the task, an error will be raised:

In [16]:
task3a()

TypeError: unsupported operand type(s) for +: 'int' and '_Nothing'

## Output directory and caching the results

After running the task, we can check where the output directory with the results was created:

In [17]:
task3.output_dir

PosixPath('/tmp/tmp0zu9bo2v/FunctionTask_2f1205974a3ee6dc201f80f3cbb600f54aeb924d9404e6d4c19457a89312229f')

Within the directory you can find the file with the results: `_result.pklz`.

In [18]:
import os

In [19]:
os.listdir(task3.output_dir)

['_task.pklz', '_result.pklz']

But we can also provide the path where we want to store the results. If a path is provided for the cache directory, then pydra will use the cached results of a node instead of recomputing the result. Let's create a temporary directory and a specific subdirectory "task4":

In [20]:
from tempfile import mkdtemp
from pathlib import Path

In [21]:
cache_dir_tmp = Path(mkdtemp()) / 'task4'
print(cache_dir_tmp)

/tmp/tmpd2ug0b3w/task4


Now we can pass this path to the argument of `FunctionTask` - `cache_dir`. To observe the execution time, we specify a function that is sleeping for 5s:

In [22]:
@pydra.mark.task
def add_var_wait(a, b):
    import time

    time.sleep(5)
    return a + b


task4 = add_var_wait(a=4, b=6, cache_dir=cache_dir_tmp)

If you're running the cell first time, it should take around 5s.

In [23]:
task4()
task4.result()

Result(output=Output(out=10), runtime=None, errored=False)

We can check `output_dir` of our task, it should contain the path of `cache_dir_tmp` and the last part contains the name of the task class `FunctionTask` and the task checksum:

In [24]:
task4.output_dir

PosixPath('/tmp/tmpd2ug0b3w/task4/FunctionTask_4cfed14a91199653af7c32b2d9c696bc02778c538d8176f61da581ec020b60fd')

Let's see what happens when we defined identical task again with the same `cache_dir`:

In [25]:
task4a = add_var_wait(a=4, b=6, cache_dir=cache_dir_tmp)
task4a()

Result(output=Output(out=10), runtime=None, errored=False)

This time the result should be ready right away! *pydra* uses available results and do not recompute the task.

*pydra* not only checks for the results in `cache_dir`, but you can provide a list of other locations that should be checked. Let's create another directory that will be used as `cache_dir` and previous working directory will be used in `cache_locations`.

In [26]:
cache_dir_tmp_new = Path(mkdtemp()) / 'task4b'

task4b = add_var_wait(
    a=4, b=6, cache_dir=cache_dir_tmp_new, cache_locations=[cache_dir_tmp]
)
task4b()

Result(output=Output(out=10), runtime=None, errored=False)

This time the results should be also returned quickly! And we can check that `task4b.output_dir` was not created:

In [27]:
task4b.output_dir.exists()

False

If you want to rerun the task regardless having already the results, you can set `rerun` to `True`. The task will take several seconds and new `output_dir` will be created:

In [28]:
cache_dir_tmp_new = Path(mkdtemp()) / 'task4c'

task4c = add_var_wait(
    a=4, b=6, cache_dir=cache_dir_tmp_new, cache_locations=[cache_dir_tmp]
)
task4c(rerun=True)

task4c.output_dir.exists()

True

If we update the input of the task, and run again, the new directory will be created and task will be recomputed:

In [29]:
task4b.inputs.a = 1
print(task4b())
print(task4b.output_dir.exists())

Result(output=Output(out=7), runtime=None, errored=False)
True


and when we check the `output_dir`, we can see that it's different than last time:

In [30]:
task4b.output_dir

PosixPath('/tmp/tmpytwxt6um/task4b/FunctionTask_c2ac026fb3cfbc88cecadfefc0a4997184e14cfa4dd2de05025edcd22d8c70e4')

This is because, the checksum changes when we change either input or function.

### Exercise 1
Create a task that take a list of numbers as an input and returns two fields: `mean` with the mean value and `std` with the standard deviation value.

In [31]:
@pydra.mark.task
@pydra.mark.annotate({'return': {'mean': ty.Any, 'std': ty.Any}})
def mean_dev(my_list):
    import statistics as st

    return st.mean(my_list), st.stdev(my_list)


my_task = mean_dev(my_list=[2, 2, 2])
my_task()
my_task.result()

Result(output=Output(mean=2, std=0.0), runtime=None, errored=False)

In [32]:
# write your solution here (you can use statistics module)

## Using Audit

*pydra* can record various run time information, including the workflow provenance, by setting `audit_flags` and the type of messengers.

`AuditFlag.RESOURCE` allows you to monitor resource usage for the `Task`, while `AuditFlag.PROV` tracks the provenance of the `Task`.

In [33]:
from pydra.utils.messenger import AuditFlag, PrintMessenger

task5 = add_var(a=4, b=5, audit_flags=AuditFlag.RESOURCE)
task5()
task5.result()

Result(output=Output(out=9), runtime=Runtime(rss_peak_gb=0.084354400390625, vms_peak_gb=0.82459258984375, cpu_peak_percent=258.7), errored=False)

One can turn on both audit flags using `AuditFlag.ALL`, and print the messages on the terminal using the `PrintMessenger`.

In [34]:
task5 = add_var(
    a=4, b=5, audit_flags=AuditFlag.ALL, messengers=PrintMessenger()
)
task5()
task5.result()

id: 01f2632a263747d6b0be6038dd975c33
{
  "@context": "https://raw.githubusercontent.com/nipype/pydra/master/pydra/schema/context.jsonld",
  "@id": "uid:df8f059181ec4bb5bdd451fd2b28fca2",
  "@type": "task",
  "startedAtTime": "2023-10-16T17:59:24.108005",
  "executedBy": "uid:930eeb6fcaff42229a33599a82bc7f84"
}


id: cbb78b7e29e64f318b1f8c22f41293d7
{
  "@context": "https://raw.githubusercontent.com/nipype/pydra/master/pydra/schema/context.jsonld",
  "@id": "uid:df8f059181ec4bb5bdd451fd2b28fca2",
  "@type": "task",
  "Label": "add_var",
  "Command": null,
  "StartedAtTime": "2023-10-16T17:59:24.312235",
  "AssociatedWith": null
}
id: c1af93a95a5c43d69fc5b8dd20f28fcb
{
  "@context": "https://raw.githubusercontent.com/nipype/pydra/master/pydra/schema/context.jsonld",
  "@id": "uid:05085826aab64257be327ddffb559886",
  "@type": "monitor",
  "startedAtTime": "2023-10-16T17:59:24.312787",
  "wasStartedBy": "uid:df8f059181ec4bb5bdd451fd2b28fca2"
}
id: d76ea6e9930c46cf94bcbf3a976e83eb
{
  "@context": "https://raw.githubusercontent.com/nipype/pydra/master/pydra/schema/context.jsonld",
  "@id": "uid:05085826aab64257be327ddffb559886",
  "endedAtTime": "2023-10-16T17:59:24.346904",
  "wasEndedBy": "uid:df8f059181ec4bb5bdd451fd2b28fca2"
}
id: 85680b61b3ca4255aeb55e9c4fa46cc6
{
  "@context": "https://raw.git

Result(output=Output(out=9), runtime=Runtime(rss_peak_gb=0.0826950078125, vms_peak_gb=0.82459258984375, cpu_peak_percent=94.2), errored=False)